<a href="https://colab.research.google.com/github/k24d/colab/blob/main/7_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3 prefect~=0.13

In [ ]:
# p.xx List 7.7

from prefect import task, Flow

@task
def get_message():
    return 'Hello, world!'

@task
def print_message(msg):
    print(msg)

with Flow('hello') as flow:
    msg = get_message()
    print_message(msg)

if __name__ == '__main__':
    flow.run()

In [ ]:
# p.xx List 7.8

import prefect
from prefect import task, Flow, Parameter

@task
def download(date):
    target_date = date or prefect.context.yesterday
    print(f"Donwload ftp://server/{target_date}/app.log")

with Flow('download') as flow:
    date = Parameter('date', default=None)
    download(date)

if __name__ == '__main__':
    flow.run()

In [ ]:
# p.xx List 7.10

from prefect import Task, Flow

class LogTask(Task):
    def __init__(self, prefix, **kwargs):
        super().__init__(**kwargs)
        self.prefix = prefix

    def run(self, message):
        print(f"{self.prefix}: {message}")

log = LogTask('INFO')

with Flow('task-class') as flow:
    log('message')

if __name__ == '__main__':
    flow.run()

In [ ]:
# Enable pytest in notebooks
!pip install ipytest
import ipytest
ipytest.autoconfig()

In [ ]:
# p.xx List 7.11
%%run_pytest

from prefect import task, Flow

@task
def get_message():
    return 'Hello, world!'

def test_get_message():
    with Flow('test') as flow:
        get_message()
    state = flow.run()
    task = flow.get_tasks(name='get_message')[0]
    assert state.result[task].result == 'Hello, world!'

In [ ]:
# p.xx List xx
# NOTE: This doesn't work on Colab because docker is not installed.

from os.path import abspath
from prefect.environments.storage import Docker

storage = Docker(
    base_image='ghcr.io/wdpressplus-bigdata/spark:latest',
    image_name='warehouse',
    # registry_url='ghcr.io/MY-PROJECT',
    files={
      abspath('scripts/warehouse.py'): '/opt/scripts/warehouse.py',
    },
)
storage.add_flow(flow)
storage.build()